In [1]:
@info("Loading packages")
using Pkg; Pkg.activate("../test")
using Plots, StatsPlots, Plots.Measures
using StatsBase
using Glob
using DataFrames, DataFramesMeta
using ProgressMeter
default(leg = false, lw = 1.5)

using ComponentArrays, StaticArrays
using Test
using Parameters
using Distributions
using OrdinaryDiffEq

# establishing type hierarchy
abstract type AbstractParams end
abstract type AbstractParamCollection end # an AbstractParamCollection contain a defined set of multiple AbstractParams instances
abstract type AbstractSpeciesParams <: AbstractParams end
abstract type AbstractGlobalODEParams <: AbstractParams end
abstract type AbstractABM end
abstract type AbstractAgent end

include("../src/DoseResponse.jl")
include("../src/Params.jl")
include("../src/Solvers.jl")
include("../src/StateVars.jl")
include("../src/IO.jl")
include("../src/ModelFunctions.jl")
include("../src/DEBODE.jl")
include("../src/ImpliedTraits.jl")
include("../src/Macros.jl")

┌ Info: Loading packages
└ @ Main c:\Users\simon\Documents\Julia\Hydra\test\abm.ipynb:1
  Activating project at `c:\Users\simon\Documents\Julia\Hydra\test`


@replicates

## ABM

In [3]:
abstract type AbstractABM end

"""
Currently assumes that all agents are represented by a single agent type `AgentType`.
"""
mutable struct ABM <: AbstractABM
    p::AbstractParamCollection # parameter collection
    t::Float64 # current simulation time
    dt::Float64 # timestep
    euler::Function # definition of the euler function for the given dt
    saveat::Float64
    
    u::ComponentVector # global state variables
    du::ComponentVector # global derivatives

    AgentID_count::Int64 # cumulative number of agents in the simulation
    agents::AbstractVector # population of agents
    aout::AbstractVector # agent output
    mout::AbstractVector # model output

    """
    Instantiate ABM from param collection `p`. 
    """
    function ABM(
        p::A; 
        dt = 1/24, 
        saveat = 1,
        execute = true
        ) where A <: AbstractParamCollection

        abm = new() # initialize ABM object

        abm.p = p # store the parameters
        abm.t = 0. # initialize simulation time
        abm.dt = dt # timestep is given as keyword argument
        abm.euler = defeuler(dt) # Euler function for the given dt
        abm.saveat = saveat
        
        abm.u = init_substates_global(p) # initialize the global substates
        abm.du = similar(abm.u) # initialize the global derivatives
        abm.du.X_p = 0.
        abm.du.C_W .= 0.
        
        abm.AgentID_count = 0 # set agent count
        initialize_agents!(abm) # initailize the population of agents
        abm.aout = [] # initialize agent output
        abm.mout = [] # initialize model output

        if execute
            run!(abm) # run the model

            return prepare_output(abm) # prepare output in dataframes and return
        else
            return abm
        end
    end
end

ABM

In [4]:
using Random

"""
step!(abm::ABM; showprogress = false)

Execution of a generic ABM step, following the schedule: 

1. Shuffle agents
2. Calculate global derivatives
3. Calculate agent derivatives
4. Update agent states
5. Record agent states
6. Update global states
7. Record global states

It is important that the agent steps occur between calculation of the global derivatives and 
updating the global states, because global derivatives which may be influenced by agent derivatives 
are initialized during calculation of the global states and may be mutated by the agent step. <br>
Changing this order will lead to erroneous computation of the interaction between agents and the environment.

---

args

-


---

kwargs 

- showprogress: shows a progress

"""
function step!(abm::ABM; showprogress = false)

    du, u, p = abm.du, abm.u, abm.p
    t = abm.t

    shuffle!(abm.agents)

    for func! in abm.p.glb.odefuncs # execute global ODE-based functions
        func!(du, u, p, t)
    end

    for func! in abm.p.glb.rulefuncs # execute global rule-based functions
        func!(abm)
    end
    
    for a in abm.agents # for every agent
        step!(a, abm) # execute agent steps
        record!(a, abm) # record agent data
    end

    map!(abm.euler, u, du, u) # apply the euler scheme to global states
    record!(abm) # record global states
    filter!(a -> a.u.agn.dead == false, abm.agents) # remove dead agents

    abm.t += abm.dt
end


step!

In [5]:
"""
    init_agents!(abm::AbstractABM)::nothing
Initialize the population of 
"""
function initialize_agents!(abm::AbstractABM)::Nothing

    abm.agents = [] # initialize a vector of agents with undefined values and defined length

    for i in 1:abm.p.glb.N0 # for the number of initial agents
        
        push!(abm.agents, abm.p.glb.AgentType(abm)) # initialize an agent and add it to the vector of agents
    end

    return nothing
end

initialize_agents!

## Agent

In [6]:
abstract type AbstractAgent end
import Base:copy


"""
The definition of a generic AbstractAgent.
"""
mutable struct DEBAgent <: AbstractAgent
    p::AbstractParamCollection
    u::ComponentVector
    du::ComponentVector
    AgentID::Int64
    adata_t::AbstractVector

    """
        DEBAgent(abm::ABM)
    
    Initialization of a generic DEB agent within ABM structure.
    """
    function DEBAgent(abm::ABM)
        a = new()

        a.p = copy(abm.p) # parameters; TODO: #29 avoid copying everything
        a.p.agn = AgentParams(a.p.spc) # assign agent-level parameters (induces individual variability)
        initialize_statevars!(a, abm) # initialize agent-level state variables
        a.du = similar(a.u) # initialize agent-level derivatives
        a.du.glb = abm.du # reference to global derivatives
        a.du.agn = similar(a.u.agn) # derivatives of agent substates have same shape as states
        a.AgentID = abm.AgentID_count # assign AgentID
        abm.AgentID_count += 1 # increment AgentID counter

        aout_t = Vector{Any}(undef, length(abm.p.spc.adata)) #
        
        return a
    end
end

DEBAgent

In [15]:
"""
Update an agent's reference to global state variables.
"""
function update_glb!(agent::AbstractAgent, abm::AbstractABM)
    map!(x -> x, agent.u.glb, abm.u) 
end

"""
step!(agent::AbstractAgent, abm::AbstractABM; odefuncs::Vector{Function}, rulefuncs::Vector{Function})

Generic definition of agent step. \n
This definition is generic, so that the function body does not have to be modified 
in order to simulate different models.
"""
function step!(agent::AbstractAgent, abm::ABM)
    du, u, p = agent.du, agent.u, agent.p # unpack agent substates, derivatives and parameters
    t = abm.t

    update_glb!(agent, abm) # update references to global state variables
    
    for func! in agent.p.spc.rulefuncs # apply all rule-based functions
        func!(agent, abm)
    end

    for func! in agent.p.spc.odefuncs # apply all ODE-based functions
        func!(du, u, p, t) 
    end


    map!(abm.euler, u.agn, du.agn, u.agn) # apply the euler scheme to agent substates
end

"""
record!(agent::DEBAgent, abm::ABM)

Record agent output (only if `p.glb.recordagentvars == true`).
"""
function record!(agent::AbstractAgent, abm::ABM)
    if abm.p.glb.recordagentvars && isapprox(abm.t%abm.saveat, 0, atol = abm.dt)
        push!(abm.aout, (t = abm.t, AgentID = agent.AgentID, u = deepcopy(agent.u.agn)))
    end
end


record!

In [20]:
using ProgressMeter

"""
record!(abm::ABM)

Record model-level output.
"""
function record!(abm::ABM)
    if isapprox(abm.t%abm.saveat, 0, atol = abm.dt)
        push!(abm.mout, (t = abm.t, u = deepcopy(abm.u)))
    end
end

"""
Run the ABM.
"""
function run!(abm::AbstractABM)
    while abm.t <= abm.p.glb.t_max
        step!(abm)
    end
end

function runprogress!(abm::AbstractABM)
    num_steps = ceil(abm.t / abm.dt)

    @showprogress for step in num_steps
        step!(abm)
    end
end

run!

In [21]:
"""
    prepare_output(abm::AbstractABM)

Prepare ABM output, converting Vectors of ComponentArrays into DataFrames. 
Two separate DataFrames are returned containing global and agent-level state variables, respectively.
"""
function prepare_output(abm::AbstractABM)

    mout = DataFrame(hcat([[x.t] for x in abm.mout]...)', [:t]) |> 
    x-> hcat(x, DataFrame(hcat([m.u for m in abm.mout]...)', extract_colnames(abm.mout[1].u)))
    
    if abm.p.glb.recordagentvars
        aout = DataFrame(hcat([[x.t, x.AgentID] for x in abm.aout]...)', [:t, :AgentID]) |> 
        x-> hcat(x, DataFrame(hcat([a.u for a in abm.aout]...)', extract_colnames(abm.aout[1].u)))
    else
        aout = DataFrame()
    end

    return mout, aout
end

prepare_output

In [22]:
theta = ABMParamCollection(
    glb = GlobalABMParams(AgentType = DEBAgent)
)


ABMParamCollection
  glb: GlobalABMParams
  spc: SpeciesParams
  agn: Nothing nothing


In [ ]:
Hydra.Adot!

In [ ]:
Hydra.Adot!

In [26]:
theta.glb.N0 = 10.
theta.glb.t_max = 56.
theta.glb

GlobalABMParams
  N0: Int64 10
  t_max: Float64 56.0
  Xdot_in: Float64 1200.0
  k_V: Float64 0.1
  V_patch: Float64 0.05
  C_W: Array{Float64}((1,)) [0.0]
  AgentType: DEBAgent <: AbstractAgent
  recordagentvars: Bool true
  saveat: Float64 1.0
  odefuncs: Array{Function}((2,))
  rulefuncs: Array{Function}((1,))


In [36]:
using Pkg; Pkg.add("NamedArrays")

   Resolving package versions...
  No Changes to `C:\Users\simon\Documents\Julia\Hydra\test\Project.toml`
  No Changes to `C:\Users\simon\Documents\Julia\Hydra\test\Manifest.toml`


In [37]:
using NamedArrays

In [50]:
nmdar = NamedArray([1, 2, 3]; names = (["a", "b", "c"],))
nmdar

3-element Named Vector{Int64}
A  │ 
───┼──
a  │ 1
b  │ 2
c  │ 3

In [51]:
nmdar |> typeof |> fieldnames

(:array, :dicts, :dimnames)

In [57]:
theta.spc.odefuncs[end] |> less

end

@inline function age!(
    du::ComponentVector,
    u::ComponentVector,
    p::AbstractParamCollection,
    t::Real
    )::Nothing 

    du.agn.age = 1.

    return nothing
end

"""
Response to chemical stressors, assuming independent action for mixtures.
"""
@inline function y_z!(
    du::ComponentVector,
    u::ComponentVector,
    p::AbstractParamCollection,
    t::Real
    )::Nothing 

    @inbounds u.agn.y_G = prod([p.spc.drc_functs_G[z](u.agn.D_G[z], (p.spc.e_G[z], p.spc.b_G[z])) for z in 1:length(u.glb.C_W)]) # combined relative responses for sublethal effects per PMoA
    @inbounds u.agn.y_M = prod([p.spc.drc_functs_M[z](u.agn.D_M[z], (p.spc.e_M[z], p.spc.b_M[z])) for z in 1:length(u.glb.C_W)])
    @inbounds u.agn.y_A = prod([p.spc.drc_functs_A[z](u.agn.D_A[z], (p.spc.e_A[z], p.spc.b_A[z])) for z in 1:length(u.glb.C_W)])
    @inbounds u.agn.y_R = prod([p.spc.drc_functs_R[z](u.agn.D_R[z], (p.spc.e_R[z], p.spc.b_R[z])) for z in 1:length(u.glb.C_W)])

    @inbounds u.agn.h_z 

In [60]:

less(Hydra.DEBODE!)

function DEBODE!(du, u, p, t)::Nothing

    #### physiological responses
    y_z!(du, u, p, t) # response to chemical stressors
    h_S!(du, u, p, t) # starvation mortality

    #### auxiliary state variables (record cumulative values)
    Idot!(du, u, p, t)
    Adot!(du, u, p, t) 
    Mdot!(du, u, p, t) 
    Jdot!(du, u, p, t)
    Qdot!(du, u, p, t)

    #### major state variables
    Sdot!(du, u, p, t) # structure
    S_0dot!(du, u, p, t) # reference structure
    Hdot!(du, u, p, t) # maturity 
    H_bdot!(du, u, p, t) # estimate of maturity at birth
    Rdot!(du, u, p, t) # reproduction buffer
    X_pdot!(du, u, p, t) # resource abundance
    X_embdot!(du, u, p, t) # vitellus
    Ddot!(du, u, p, t) # damage
    C_Wdot!(du, u, p, t) # external stressor concentration 

    return nothing
end


In [27]:
Yhat = ABM(theta)

DomainError: DomainError with -0.0026640581113757888:
log was called with a negative real argument but will only return a complex result if called with a complex argument. Try log(Complex(x)).
DomainError detected in the user `f` function. This occurs when the domain of a function is violated.
For example, `log(-1.0)` is undefined because `log` of a real number is defined to only output real
numbers, but `log` of a negative number is complex valued and therefore Julia throws a DomainError
by default. Cases to be aware of include:

* `log(x)`, `sqrt(x)`, `cbrt(x)`, etc. where `x<0`
* `x^y` for `x<0` floating point `y` (example: `(-1.0)^(1/2) == im`)

Within the context of SciML, this error can occur within the solver process even if the domain constraint
would not be violated in the solution due to adaptivity. For example, an ODE solver or optimization
routine may check a step at `new_u` which violates the domain constraint, and if violated reject the
step and use a smaller `dt`. Howeve